### Imports + Data Cleaning

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import tensorflow as tf
from tensorflow import keras
from transformers import BertTokenizer, TFBertModel
import math

2023-11-21 20:19:13.714371: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-21 20:19:19.134242: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-21 20:19:19.157183: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 20:19:30.840054: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def parse_jsonl(path: str) -> pd.DataFrame:
    data = {}
    with open(path, 'r') as f:
        for (i, line) in enumerate(f):
            line = json.loads(line)
            line["conversation_id"] = [i] * len(line["messages"])
            for k in line.keys():
                if k == "players":
                    continue
                if k == "game_id":
                    line[k] = [line[k]] * len(line["messages"])
                data[k] = data.get(k, []) + line[k]
    return pd.DataFrame(data)

In [3]:
train_df = pd.read_json("./data/train.jsonl", lines=True)
train_df.head()

,messages,sender_labels,receiver_labels,speakers,receivers,absolute_message_index,relative_message_index,seasons,years,game_score,game_score_delta,players,game_id
0,[Germany!\n\nJust the person I want to speak w...,"[True, True, True, True, True, True, True, Tru...","[True, True, True, True, NOANNOTATION, NOANNOT...","[italy, germany, italy, germany, italy, italy,...","[germany, italy, germany, italy, germany, germ...","[74, 76, 86, 87, 89, 92, 97, 117, 119, 121, 12...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[Spring, Spring, Spring, Spring, Spring, Sprin...","[1901, 1901, 1901, 1901, 1901, 1901, 1901, 190...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[italy, germany]",1
1,[Hello there! What's your general plan for thi...,"[True, False, True, False, True, True, True, T...","[True, True, True, True, True, NOANNOTATION, T...","[austria, italy, austria, italy, italy, austri...","[italy, austria, italy, austria, austria, ital...","[1, 67, 71, 73, 98, 99, 101, 179, 181, 185, 18...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[Spring, Spring, Spring, Spring, Spring, Sprin...","[1901, 1901, 1901, 1901, 1901, 1901, 1901, 190...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 5, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, -1, -...","[italy, austria]",1
2,[Buongiorno! \nBe kinda nice to know if you're...,"[True, True, False, True, True, True, True, Tr...","[True, False, True, False, True, True, NOANNOT...","[russia, italy, russia, italy, russia, italy, ...","[italy, russia, italy, russia, italy, russia, ...","[11, 50, 52, 57, 61, 66, 77, 85, 96, 102, 116,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[Spring, Spring, Spring, Spring, Spring, Sprin...","[1901, 1901, 1901, 1901, 1901, 1901, 1901, 190...","[4, 3, 4, 3, 4, 3, 4, 3, 3, 3, 4, 3, 3, 4, 4, ...","[1, -1, 1, -1, 1, -1, 1, -1, -1, -1, 1, -1, -1...","[italy, russia]",1
3,[Hey italy! good luck this game. I'm guessing ...,"[True, False, True, True, True, True, True, Tr...","[NOANNOTATION, True, True, False, True, True, ...","[england, italy, england, england, england, it...","[italy, england, italy, italy, italy, england,...","[32, 95, 106, 107, 108, 110, 113, 125, 126, 12...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[Spring, Spring, Spring, Spring, Spring, Sprin...","[1901, 1901, 1901, 1901, 1901, 1901, 1901, 190...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[italy, england]",1
4,[Hello Italy what’s up what are your thoughts ...,"[True, False, False, True, True, True, True, T...","[NOANNOTATION, True, True, True, True, True, N...","[turkey, italy, italy, italy, turkey, italy, t...","[italy, turkey, turkey, turkey, italy, turkey,...","[45, 94, 103, 150, 154, 178, 192, 194, 195, 19...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[Spring, Spring, Spring, Spring, Fall, Fall, F...","[1901, 1901, 1901, 1901, 1901, 1901, 1901, 190...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 1...","[italy, turkey]",1


In [4]:
train_df = (parse_jsonl("./data/train.jsonl"))
test_df = (parse_jsonl("./data/test.jsonl"))
validation_df = (parse_jsonl("./data/validation.jsonl"))

In [5]:
print(train_df.shape)
print(test_df.shape)
print(validation_df.shape)

(13132, 13)
(2741, 13)
(1416, 13)


In [6]:
train_df.head()

,messages,sender_labels,receiver_labels,speakers,receivers,absolute_message_index,relative_message_index,seasons,years,game_score,game_score_delta,game_id,conversation_id
0,Germany!\n\nJust the person I want to speak wi...,True,True,italy,germany,74,0,Spring,1901,3,0,1,0
1,"You've whet my appetite, Italy. What's the sug...",True,True,germany,italy,76,1,Spring,1901,3,0,1,0
2,👍,True,True,italy,germany,86,2,Spring,1901,3,0,1,0
3,It seems like there are a lot of ways that cou...,True,True,germany,italy,87,3,Spring,1901,3,0,1,0
4,"Yeah, I can’t say I’ve tried it and it works, ...",True,NOANNOTATION,italy,germany,89,4,Spring,1901,3,0,1,0


In [7]:
def determine_name(row):
    if row['sender_labels'] and row['receiver_labels']:
        return 'straightforward'
    elif not row['sender_labels'] and row['receiver_labels']:
        return 'deceive'
    elif row['sender_labels'] and not row['receiver_labels']:
        return 'cassandra'
    elif not row['sender_labels'] and not row['receiver_labels']:
        return 'caught'
    else:
        return None

In [8]:
train_df['label'] = train_df.apply(determine_name, axis=1)

In [9]:
test_df['label'] = test_df.apply(determine_name, axis=1)

In [10]:
train_df

,messages,sender_labels,receiver_labels,speakers,receivers,absolute_message_index,relative_message_index,seasons,years,game_score,game_score_delta,game_id,conversation_id,label
0,Germany!\n\nJust the person I want to speak wi...,True,True,italy,germany,74,0,Spring,1901,3,0,1,0,straightforward
1,"You've whet my appetite, Italy. What's the sug...",True,True,germany,italy,76,1,Spring,1901,3,0,1,0,straightforward
2,👍,True,True,italy,germany,86,2,Spring,1901,3,0,1,0,straightforward
3,It seems like there are a lot of ways that cou...,True,True,germany,italy,87,3,Spring,1901,3,0,1,0,straightforward
4,"Yeah, I can’t say I’ve tried it and it works, ...",True,NOANNOTATION,italy,germany,89,4,Spring,1901,3,0,1,0,straightforward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13127,Is there any way of me actually ending this co...,True,True,france,england,380,11,Fall,1902,4,-1,10,187,straightforward
13128,Can we agree on peace? What are your demands?,True,True,france,england,433,12,Fall,1902,4,-1,10,187,straightforward
13129,"Neutrality in exchange for current holdings, S...",True,False,england,france,434,13,Fall,1902,5,1,10,187,cassandra
13130,"Thats a bit too much, can I keep Spain and i h...",True,True,france,england,437,14,Fall,1902,4,-1,10,187,straightforward


### Data Exploration

In [11]:
train_df['sender_labels'].unique()

array([ True, False])

In [12]:
train_df['receiver_labels'].unique()

array([True, 'NOANNOTATION', False], dtype=object)

In [13]:
train_df['game_score_delta'].unique()

array(['0', '1', '-1', '2', '-2', '3', '-3', '6', '-6', '7', '-7', '4',
       '-4', '10', '-10', '5', '-5', '8', '-8', '-9', '9', '13', '-13',
       '12', '11', '-11', 0, '14', '-14'], dtype=object)

# NONANNOTATION

We can see that there are some `receiver_labels` that have the value `NOANNOTATION`. this is not a bool value 

In [14]:
round(len(train_df[train_df["receiver_labels"] == "NOANNOTATION"].index) /  len(train_df.index), 4) * 100

8.43

In [15]:
round(len(train_df[train_df["sender_labels"] == "NOANNOTATION"].index) /  len(train_df.index), 4) * 100

0.0

In [16]:
train_df.head(10)['messages'][0]

'Germany!\n\nJust the person I want to speak with. I have a somewhat crazy idea that I’ve always wanted to try with I/G, but I’ve never actually convinced the other guy to try it. And, what’s worse, it might make you suspicious of me. \n\nSo...do I suggest it?\n\nI’m thinking that this is a low stakes game, not a tournament or anything, and an interesting and unusual move set might make it more fun? That’s my hope anyway.\n\nWhat is your appetite like for unusual and crazy?'

### Baseline: Implement a Naive Bayes Model w/ Bag of Words

In [17]:
import sklearn

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import imblearn

In [19]:
X_train, X_test, y_train, y_test = train_df['messages'], test_df['messages'], train_df['label'], test_df['label']

## Process X train and X test
count_vect = CountVectorizer()
X_train_c = count_vect.fit_transform(X_train)
X_test_c = count_vect.transform(X_test)

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_c)
X_train_tf = tf_transformer.transform(X_train_c)
X_test_tf = tf_transformer.transform(X_test_c)

from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train_tf_smote, y_train_df_smote = oversample.fit_resample(X_train_tf, y_train)

## Final Sanity Check
print(f"Train set size: {X_train_tf_smote.shape}")
print(f"Test set size: {X_test_tf.shape}")

Train set size: (48160, 7753)
Test set size: (2741, 7753)


In [20]:
## Train A Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tf_smote, y_train_df_smote)

In [21]:
## Check Performance on Train and Test Set
from sklearn import metrics
y_train_pred = clf.predict(X_train_tf)
y_test_pred = clf.predict(X_test_tf)
print("Train Metrics")
print(metrics.classification_report(y_train, y_train_pred))

print("Test Metrics")
print(metrics.classification_report(y_test, y_test_pred))

Train Metrics
                 precision    recall  f1-score   support

      cassandra       0.16      0.80      0.27       501
         caught       0.07      0.95      0.12        65
        deceive       0.17      0.82      0.28       526
straightforward       0.99      0.58      0.73     12040

       accuracy                           0.60     13132
      macro avg       0.35      0.79      0.35     13132
   weighted avg       0.92      0.60      0.69     13132

Test Metrics
                 precision    recall  f1-score   support

      cassandra       0.06      0.27      0.10       123
         caught       0.03      0.14      0.05        42
        deceive       0.11      0.34      0.17       198
straightforward       0.89      0.51      0.65      2378

       accuracy                           0.48      2741
      macro avg       0.27      0.31      0.24      2741
   weighted avg       0.78      0.48      0.58      2741



### LSTM

In [40]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df['messages'])

train_sequences = tokenizer.texts_to_sequences(train_df['messages'])
train_padded = pad_sequences(train_sequences, padding='post')

test_sequences = tokenizer.texts_to_sequences(test_df['messages'])
test_padded = pad_sequences(test_sequences, padding='post', maxlen=train_padded.shape[1])

#val_sequences = tokenizer.texts_to_sequences(validation_df['messages'])
#val_padded = pad_sequences(val_sequences, padding='post', maxlen=train_padded.shape[1])


In [41]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
label_encoder.fit(train_df['label'])  # Fit on training labels

# Transform labels in all datasets
train_labels = to_categorical(label_encoder.transform(train_df['label']))
test_labels = to_categorical(label_encoder.transform(test_df['label']))
#val_labels = to_categorical(label_encoder.transform(validation_df['label']))


In [46]:
oversample = SMOTE()
train_padded_smote, train_labels_smote = oversample.fit_resample(train_padded, train_labels)

In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64))
model.add(LSTM(128))
model.add(Dense(10, activation='relu'))
model.add(Dense(4, activation='softmax'))  # Assuming you have 4 labels


2023-11-21 21:50:40.270852: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-21 21:50:40.272842: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-21 21:50:40.274396: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [53]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [54]:
model.fit(train_padded_smote, train_labels_smote, epochs=10, batch_size=32, validation_data=(test_padded, test_labels))

Epoch 1/10


2023-11-21 21:50:42.647441: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-21 21:50:42.649745: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-21 21:50:42.651587: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1505/1505 [==============================] - ETA: 0s - loss: 1.3863 - accuracy: 0.2499

2023-11-21 21:58:09.684211: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-21 21:58:09.686411: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-21 21:58:09.688114: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1505/1505 [==============================] - 458s 303ms/step - loss: 1.3863 - accuracy: 0.2499 - val_loss: 1.3811 - val_accuracy: 0.0449
Epoch 2/10
1505/1505 [==============================] - 452s 301ms/step - loss: 1.3864 - accuracy: 0.2468 - val_loss: 1.3867 - val_accuracy: 0.0722
Epoch 3/10
1505/1505 [==============================] - 453s 301ms/step - loss: 1.3864 - accuracy: 0.2482 - val_loss: 1.3762 - val_accuracy: 0.8676
Epoch 4/10
1505/1505 [==============================] - 452s 300ms/step - loss: 1.3864 - accuracy: 0.2464 - val_loss: 1.3832 - val_accuracy: 0.0722
Epoch 5/10
1505/1505 [==============================] - 452s 300ms/step - loss: 1.3864 - accuracy: 0.2494 - val_loss: 1.3691 - val_accuracy: 0.8676
Epoch 6/10
1505/1505 [==============================] - 456s 303ms/step - loss: 1.3864 - accuracy: 0.2484 - val_loss: 1.3894 - val_accuracy: 0.0153
Epoch 7/10
1505/1505 [==============================] - 454s 302ms/step - loss: 1.3864 - accuracy: 0.2519 - val_loss: 1.389

In [55]:
loss, accuracy = model.evaluate(test_padded, test_labels)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


86/86 [==============================] - 7s 77ms/step - loss: 1.3765 - accuracy: 0.8676
Test Loss: 1.3764708042144775, Test Accuracy: 0.8675665855407715


In [59]:
y_train_pred = model.predict(train_padded)
y_test_pred = model.predict(test_padded)


86/86 [==============================] - 6s 74ms/step


In [60]:
import numpy as np

# Convert one-hot encoded predictions to class labels
y_train_pred_labels = np.argmax(y_train_pred, axis=1)
y_test_pred_labels = np.argmax(y_test_pred, axis=1)

# Similarly, convert the true labels from one-hot encoded form to class labels
y_train_true_labels = np.argmax(train_labels, axis=1)
y_test_true_labels = np.argmax(test_labels, axis=1)


In [61]:
from sklearn.metrics import classification_report

# Print classification report for training data
print("Train Metrics")
print(classification_report(y_train_true_labels, y_train_pred_labels, target_names=label_encoder.classes_))

# Print classification report for test data
print("Test Metrics")
print(classification_report(y_test_true_labels, y_test_pred_labels, target_names=label_encoder.classes_))


Train Metrics
                 precision    recall  f1-score   support

      cassandra       0.00      0.00      0.00       501
         caught       0.00      0.00      0.00        65
        deceive       0.00      0.00      0.00       526
straightforward       0.92      1.00      0.96     12040

       accuracy                           0.92     13132
      macro avg       0.23      0.25      0.24     13132
   weighted avg       0.84      0.92      0.88     13132

Test Metrics
                 precision    recall  f1-score   support

      cassandra       0.00      0.00      0.00       123
         caught       0.00      0.00      0.00        42
        deceive       0.00      0.00      0.00       198
straightforward       0.87      1.00      0.93      2378

       accuracy                           0.87      2741
      macro avg       0.22      0.25      0.23      2741
   weighted avg       0.75      0.87      0.81      2741



/home/adam_kreitzman/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/adam_kreitzman/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/adam_kreitzman/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/hom

### Context LSTM

In [23]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate

# Text input branch
text_input = Input(shape=(None,))
text_embed = Embedding(input_dim=5000, output_dim=64)(text_input)
lstm_out = LSTM(64)(text_embed)

# Context input branch
context_input = Input(shape=context_data.shape[1:])
context_dense = Dense(64, activation='relu')(context_input)

# Concatenate branches
concatenated = Concatenate()([lstm_out, context_dense])

# Output layer
output = Dense(4, activation='softmax')(concatenated)

model = Model(inputs=[text_input, context_input], outputs=output)

2023-11-21 20:43:21.071757: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-21 20:43:21.073635: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-21 20:43:21.075077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

NameError: name 'context_data' is not defined